# Notebook to test out the Riot Games API for the backend and basic examples of what type of data visualizations we want to do on our website

## Prerequisites

### Cassiopeia which is a library to communicate with the Riot API

In [1]:
pip install cassiopeia

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/169.3 kB ? eta -:--:--
   --------------------------------------- 169.3/169.3 kB 10.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/66.4 kB ? eta -:--:--
   ---------------------------------------- 66.4/66.4 kB 3.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ------ --------------------------------- 0.4/2.6 MB 8.9 MB/s eta 0:00:01
   -------------- ------------------------- 0.9/2.6 MB 11.7 MB/s eta 0:00:01
   --------------------- ------------------

In [2]:
import cassiopeia as cass

### Set Riot API key 
This needs to be changed every 24 hours, a new one can be generated from the [dashboard on this link](https://developer.riotgames.com)

In [9]:
cass.set_riot_api_key("RGAPI-2913e682-e60b-4d24-91a2-84b7ea0e5c75")

## Summoners

In [31]:
from cassiopeia import Summoner

In [32]:
Fekir = Summoner(name="Hide on Bush", region="KR")


### Get level and many other stuff!

In [41]:
Fekir.level

749

## Matches

In [43]:
pip install sortedcontainers

Note: you may need to restart the kernel to use updated packages.


In [44]:
import random
from sortedcontainers import SortedList
import arrow


In [40]:
from cassiopeia.core import Summoner, MatchHistory, Match
from cassiopeia import Queue, Patch

In [54]:
import cassiopeia as cass
from cassiopeia import Summoner, Match
from cassiopeia.data import Queue
from collections import Counter


def print_newest_match(name: str, region: str):

    # Notice how this function never makes a call to the summoner endpoint because we provide all the needed data!

    summoner = Summoner(name=name, region=region)

    # A MatchHistory is a lazy list, meaning it's elements only get loaded as-needed.

    match_history = cass.get_match_history(
        continent=summoner.region.continent,
        puuid=summoner.puuid,
        queue=Queue.ranked_solo_fives,
    )
    # match_history = summoner.match_history

    # Load the entire match history by iterating over all its elements so that we know how long it is.
    # Unfortunately since we are iterating over the match history and accessing the summoner's champion for each match,
    # we need to know what version of the static data the champion should have. To avoid pulling many different
    # static data versions, we will instead create a {champion_id -> champion_name} mapping and just access the champion's
    # ID from the match data (which it provides directly).
    champion_id_to_name_mapping = {
        champion.id: champion.name for champion in cass.get_champions(region=region)
    }
    played_champions = Counter()
    for match in match_history:
        champion_id = match.participants[summoner].champion.id
        champion_name = champion_id_to_name_mapping[champion_id]
        played_champions[champion_name] += 1
    print("Length of match history:", len(match_history))

    # Print the aggregated champion results
    print(f"Top 10 champions {summoner.name} played:")
    for champion_name, count in played_champions.most_common(10):
        print(champion_name, count)
    print()

    match = match_history[0]
    print("Match ID:", match.id)

    p = match.participants[summoner]
    print(
        "\nSince the match was created from a matchref, we only know one participant:"
    )
    # print(p.summoner.name, 'playing', p.champion.name)
    print(
        p.summoner.region,
        p.summoner.account_id,
        p.summoner.name,
        p.summoner.id,
        p.champion.id,
    )

    print("\nNow pull the full match data by iterating over all the participants:")
    for p in match.participants:
        # print(p.summoner.name, 'playing', p.champion.name)
        print(
            p.summoner.region,
            p.summoner.account_id,
            p.summoner.name,
            p.summoner.id,
            p.champion.id,
            p.stats.win,
            p.runes.keystone.name,
        )

    print(
        "\nIterate over all the participants again and note that the data is not repulled:"
    )
    for p in match.participants:
        # print(p.summoner.name, 'playing', p.champion.name)
        print(
            p.summoner.region,
            p.summoner.account_id,
            p.summoner.name,
            p.summoner.id,
            p.champion.id,
            p.stats.win,
            p.runes.keystone.name,
        )

    print("\nBlue team won?", match.blue_team.win)
    print("Red team won?", match.red_team.win)
    print("Participants on blue team:")
    for p in match.blue_team.participants:
        print(p.summoner.name, p.champion.name)

    # Print keystone and the stat runes for each player/champion
    for p in match.participants:
        print(p.champion.name, p.runes.keystone.name, *[r.name for r in p.stat_runes])


if __name__ == "__main__":
    print_newest_match(name="biadee", region="EUNE")

Making call: https://eun1.api.riotgames.com/lol/summoner/v4/summoners/by-name/biadee
Making call: https://ddragon.leagueoflegends.com/realms/eune.json
Making call: https://ddragon.leagueoflegends.com/cdn/14.5.1/data/en_GB/championFull.json
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/1bDGS70e7KB9-Y9q_xjVzZfYfllMf9z7BijdH049z7ASHWPLpOW9mIoFcDbcFocQo4t9cJTMIJXM1A/ids?start=0&count=100&queue=420
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/EUN1_3544991404
Making call: https://ddragon.leagueoflegends.com/api/versions.json
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/EUN1_3544962266
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/EUN1_3544931357
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/EUN1_3544895952
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/EUN1_3544880206
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/EUN1_3544736345
Making c

NotFoundError: No source returned a query result!